In [1]:
import glob
import fitsio
import proplot as pplt
import numpy as np

In [2]:
import random

fnames = sorted(glob.glob("./data_final/*.fits"))
n_keep = 10

random.shuffle(fnames)

In [3]:
import tqdm

d = []
for i in tqdm.trange(n_keep):
    d.append(fitsio.read(fnames[i]))
d = np.concatenate(d)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.04it/s]


In [6]:
msk = (
    (d["flags"] == 0)
    & (d["mask_flags"] == 0)
    & (d["mdet_s2n"] > 8)
    & (d["mdet_s2n"] <= 100)    
    & (d["mdet_T_ratio"] > 0.5)
    & (d["mdet_T"] < 1.2)
    & (d["mfrac"] > 0.02)
    & (d["mfrac"] < 0.1)
)
print(np.sum(msk)/n_keep*1e4/1e6/5)
print(np.std(d["mdet_g_1"][msk]))

msk = (
    (d["flags"] == 0)
    & (d["mask_flags"] == 0)
    & (d["mdet_s2n"] > 8)
    & (d["mdet_s2n"] <= 100)    
    & (d["mdet_T_ratio"] > 0.5)
    & (d["mdet_T"] < 1.2)
    & (d["mfrac"] < 0.02)
)
print(np.sum(msk)/n_keep*1e4/1e6/5)
print(np.std(d["mdet_g_1"][msk]))

8.3412
0.3807955153671818
150.4648
0.37897596978657616


In [ ]:
keys = ["mdet_s2n", "mdet_T", "mdet_T_ratio"]
n_keys = len(keys)

fig, axs = pplt.subplots(nrows=n_keys, ncols=n_keys, share=0)
for i in range(n_keys):
    ax = axs[i, i]
    ax.hist(np.log10(d[keys[i]][msk]), bins=100, log=True)

    if i == n_keys - 1:
        ax.set_xlabel("log10[%s]" % keys[i])
        
    if i == 0:
        ax.set_ylabel("log10[%s]" % keys[i])

    for j in range(i+1, n_keys):
        ax = axs[j, i]
        ax.hexbin(
            np.log10(d[keys[i]][msk]), 
            np.log10(d[keys[j]][msk]), 
            norm="log",
        )
        
        if j == n_keys - 1:
            ax.set_xlabel("log10[%s]" % keys[i])

        if i == 0:
            ax.set_ylabel("log10[%s]" % keys[j])
        
    for j in range(0, i):
        ax = axs[j, i]
        ax.axis("off")
